In [1]:
#importation des données dvf géolocalisées en csv

import os
import s3fs
import pandas as pd
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]

fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={"endpoint_url": S3_ENDPOINT_URL}
)

with fs.open("renan/diffusion/dvf.csv", mode="rb") as f:
    df = pd.read_csv(f)

/tmp/ipykernel_43830/352770705.py:14: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [2]:
df_original=df.copy()
df=df.reset_index()

print(df.shape)

KeyboardInterrupt: 

In [4]:
#On se limite aux maisons et appartements

df = df[df['type_local'].isin(['Maison', 'Appartement'])]
print(df)
df=df.reset_index()

            index  id_mutation date_mutation  numero_disposition  \
0              17       2020-6    2020-07-02                   1   
1              31       2020-8    2020-07-01                   1   
2              32       2020-9    2020-07-02                   1   
3              36      2020-11    2020-07-03                   1   
4              37      2020-12    2020-07-02                   1   
...           ...          ...           ...                 ...   
6247210  20102723  2025-502875    2025-06-23                   1   
6247211  20102725  2025-502876    2025-06-25                   1   
6247212  20102726  2025-502877    2025-06-25                   1   
6247213  20102734  2025-502880    2025-06-27                   1   
6247214  20102735  2025-502881    2025-06-27                   1   

        nature_mutation  valeur_fonciere  adresse_numero adresse_suffixe  \
0                 Vente          77000.0           347.0             NaN   
1                 Vente        

In [23]:
comptage_df = df['id_mutation'].value_counts().reset_index()
comptage_df.columns = ['id_mutation', 'nombre_de_lignes']
print(comptage_df)

id_uniques = comptage_df[comptage_df['nombre_de_lignes'] == 1]
print(id_uniques.shape)

print(id_uniques)
df_sans_lots=df[df['id_mutation'].isin(id_uniques['id_mutation'])]
print(df_sans_lots)

         id_mutation  nombre_de_lignes
0        2021-687176              9290
1        2022-919567              7657
2        2022-909252              7477
3        2021-342324              7112
4        2020-491811              6173
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4861637 rows x 2 columns]
(4133749, 2)
         id_mutation  nombre_de_lignes
727888   2023-581082                 1
727889   2023-510332                 1
727890   2023-580244                 1
727891   2023-510334                 1
727892   2023-579686                 1
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4133749 rows x 2 colu

In [7]:
#les codes communes n'ont pas tous le même type ; certains sont des ints, d'autres des strings. On convertit l'ensemble en string pour plus de cohérence(on 
#ne peut pas faire l'opération inverse ; les codes corses commencent par 2A ou 2B !)

def inttostr(valeur):
    if isinstance(valeur, float):
        return str(valeur)
    if isinstance(valeur, int):
        return str(valeur)
    else:
        # Si c'est déjà une str, None, float, etc., on le retourne tel quel
        return valeur
print(inttostr('433'))

df.loc[:,'code_commune'] = df['code_commune'].apply(inttostr)
print(df['code_commune'])

433
0           1130
1           1451
2           1364
3           1053
4           1177
           ...  
6247210    75111
6247211    75112
6247212    75116
6247213    75113
6247214    75105
Name: code_commune, Length: 6247215, dtype: object


In [14]:
#Calcul du nombre d'annonces avec des identifiants distincts pour chaque commune

ventes_par_commune = df['code_commune'].value_counts().reset_index(name='nombre')

#Résultats : on remarque on particulier que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025(50684) !!!

print(ventes_par_commune)

print( ventes_par_commune["nombre"][ventes_par_commune["code_commune"]=='2A004'])

print(ventes_par_commune['nombre'].max())

print(ventes_par_commune['nombre'].idxmax())

print(ventes_par_commune['nombre'][ventes_par_commune['nombre'].idxmax()])


      code_commune  nombre
0            31555   50684
1             6088   46697
2            44109   30331
3            33063   30077
4            34172   28927
...            ...     ...
33864        80522       1
33865        31062       1
33866        31549       1
33867        39376       1
33868        40166       1

[33869 rows x 2 columns]
145    4880
Name: nombre, dtype: int64
50684
0
50684


In [16]:
#On récupère désormais les données de population ; cependant certaines valeurs ne sont pas renseignés. On remarque aussi que certaines communes ont des populations trop faibles
#pour que le ratio considéré ventes entre 2020 et 2050/population en 2019 ait du sens ; en particulier, la commune de Bezonvaux a zéro habitants !!! 
# On impose donc un nombre d'habitants minimum arbitraire, et on exclut les communes de populations trop faibles ou non renseignés

df_pop=pd.read_excel("popcommunes.xlsx")
print(df_pop)

cols = ["p19_pop"]
df_pop[cols].isna().sum()
lignes_na = df_pop[df_pop[cols].isna().any(axis=1)]
print(lignes_na)
print(lignes_na.shape)

       objectid  reg dep    cv codgeo                 libgeo  p13_pop  \
0        115658   52  85  8502  85062            Châteauneuf    968.0   
1        115659   26  58  5808  58300                   Urzy   1839.0   
2        115660   43  70  7012  70137  Chassey-lès-Montbozon    218.0   
3        115661   21  51  5123  51649      Vitry-le-François  13174.0   
4        115662   11  78  7811  78638         Vaux-sur-Seine   4749.0   
...         ...  ...  ..   ...    ...                    ...      ...   
34990    110251   31  62  6225  62327         Febvin-Palfart    574.0   
34991    110252   73  12  1201  12156            Montpeyroux    550.0   
34992    110253   54  86  8610  86138                Luchapt    275.0   
34993    110254   72  33  3320  33184                Générac    579.0   
34994    110255   72  24  2409  24354       La Roche-Chalais   2932.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
0        993.0   1013.0   1027.0     1056  

In [ ]:
#On joint les données en se restreignant aux communes dont on connaît les populations et de populations pas trop faibles. Le min est arbitraire est peut être changé

df_pop.rename(columns={'codgeo': 'code_commune'}, inplace=True)

ventes_par_commune_par_habitant=pd.merge(ventes_par_commune, df_pop, on='code_commune')
ventes_par_commune_par_habitant = ventes_par_commune_par_habitant[
    (ventes_par_commune_par_habitant['p19_pop'].notna()) &
    (ventes_par_commune_par_habitant['p19_pop'] > 1000)
]
ventes_par_commune_par_habitant['ventes par habitants par commune']=ventes_par_commune_par_habitant['nombre']/ventes_par_commune_par_habitant['p19_pop']

print(ventes_par_commune_par_habitant['ventes par habitants par commune'])

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].max())

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax())

print(ventes_par_commune_par_habitant['code_commune'][ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax()])

0      0.068385
1      0.032209
2      0.024849
3      0.047811
4      0.132321
         ...   
916    0.000547
917    0.000727
924    0.000529
925    0.000431
930    0.000541
Name: ventes par habitants par commune, Length: 374, dtype: float64
0.24206349206349206
22
2B150


In [ ]:
#tentative de récupération des prix au m2
#Remarque : on ne considère que le ratio valeur foncière/surface batîe de la maison ou de l'appartement, puisqu'on ignore les dépendances ou autres terrains additionnels.
#on sur-estime donc la valeur au m2. On ignore les lots(on utilise df_sans_lots qui ne contient pas les observations dont l'id_mutation apparait plusieurs fois)
#à la fois parce qu'on ne peut pas retrouver la valeur foncière des éléments individuels, et pour se concentrer sur les logements anciens
df_sans_lots = df_sans_lots.copy()
print(df_sans_lots.shape)

df_sans_lots = df_sans_lots[
    (df_sans_lots['surface_reelle_bati'].notna()) &
    (df_sans_lots['valeur_fonciere'].notna()) &
    (df_sans_lots['surface_reelle_bati'] > 10)
]



df_sans_lots['rapport valeur foncière et surface bâtie']=df_sans_lots['valeur_fonciere']/df_sans_lots['surface_reelle_bati']

print(df_sans_lots['rapport valeur foncière et surface bâtie'])

print(df_sans_lots['rapport valeur foncière et surface bâtie'].max())

print(df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax())

print(df_sans_lots.loc[df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax()])

#problème ; valeurs monstrueuses de certains logements, visiblement des immeubles entiers plutôt que des logements(ci-dessus,immeuble en construction de presque 1 milliard...)


moyenne_par_commune=df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

print(moyenne_par_commune['rapport valeur foncière et surface bâtie'].max())

print(moyenne_par_commune['code_commune'].loc[moyenne_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])



# Calcul de la médiane par commune
mediane_par_commune = df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Valeur maximale de la médiane
print(mediane_par_commune['rapport valeur foncière et surface bâtie'].max())

# Code de la commune avec la médiane maximale
print(mediane_par_commune['code_commune'][mediane_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])

# on retire désormais les 5% valeurs les plus extrêmes ; tronquage naîf en le faisant au niveau national, résultats insatisfaisants
seuil_bas = df_sans_lots['rapport valeur foncière et surface bâtie'].quantile(0.01)
seuil_haut = df_sans_lots['rapport valeur foncière et surface bâtie'].quantile(0.99)

# Filtrage du DataFrame
df_sans_lots_tronqué = df_sans_lots[
    (df_sans_lots['rapport valeur foncière et surface bâtie'] >= seuil_bas) &
    (df_sans_lots['rapport valeur foncière et surface bâtie'] <= seuil_haut)
]

# Calcul de la moyenne par commune (tronqué)
moyenne_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

# Affichage de la moyenne maximale et de la commune associée
print(moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(moyenne_par_commune_tronqué['code_commune'].loc[moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])

# Calcul de la médiane par commune (tronqué)
mediane_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Affichage de la médiane maximale et de la commune associée
print(mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(mediane_par_commune_tronqué['code_commune'][mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])


KeyboardInterrupt: 

In [ ]:
#meilleure alternative au troncage précédent ; on tronque par commune, on touve alors des médianes et moyennes très proches, maximale pour le sixième arrondissement parisien, c'est extrêmement cohérent !!!!

# Calculer les quantiles 0.025 et 0.975 par commune
quantiles_par_commune = (
    df_sans_lots
    .groupby('code_commune')['rapport valeur foncière et surface bâtie']
    .quantile([0.025, 0.975])
    .unstack()
    .reset_index()
    .rename(columns={0.025: 'quantile_025', 0.975: 'quantile_975'})
)

print(quantiles_par_commune)

# Fusionner les quantiles avec df_sans_lots
df_sans_lots_tronqué_bis = df_sans_lots.merge(
    quantiles_par_commune,
    on='code_commune',
    how='left'
)

df_sans_lots_tronqué_bis = df_sans_lots_tronqué_bis[
    (df_sans_lots_tronqué_bis['rapport valeur foncière et surface bâtie'] >= df_sans_lots_tronqué_bis['quantile_025']) &
    (df_sans_lots_tronqué_bis['rapport valeur foncière et surface bâtie'] <= df_sans_lots_tronqué_bis['quantile_975'])
]

# Calcul de la moyenne par commune (tronqué_bis)
moyenne_par_commune_tronqué_bis = df_sans_lots_tronqué_bis.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

# Affichage de la moyenne maximale et de la commune associée
print(moyenne_par_commune_tronqué_bis['rapport valeur foncière et surface bâtie'].max())
print(moyenne_par_commune_tronqué_bis['code_commune'].loc[moyenne_par_commune_tronqué_bis['rapport valeur foncière et surface bâtie'].idxmax()])

# Calcul de la médiane par commune (tronqué_bis)
mediane_par_commune_tronqué_bis = df_sans_lots_tronqué_bis.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Affichage de la médiane maximale et de la commune associée
print(mediane_par_commune_tronqué_bis['rapport valeur foncière et surface bâtie'].max())
print(mediane_par_commune_tronqué_bis['code_commune'][mediane_par_commune_tronqué_bis['rapport valeur foncière et surface bâtie'].idxmax()])

      code_commune  quantile_025  quantile_975
0             1001   1402.258065   4650.588503
1             1002    394.560643   3523.392756
2             1004   1017.798713   3943.055556
3             1005   1107.758621   4353.815194
4             1006    272.916667   2423.611111
...            ...           ...           ...
33661        30352   1797.182796   5541.040441
33662        30353   2335.533333   2335.533333
33663        30354   1462.309165   2210.739254
33664        30355   1539.274322   5654.069444
33665        30356   1488.383761   4125.932700

[33666 rows x 3 columns]
14889.448668014387
75106
14752.04918032787
75106
